# Bert classification

In [1]:
!pip install AugmentedSocialScientist pandas numpy

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [3]:
import pandas as pd
simplify_labels = False

annotations_file = pd.read_csv("../data/intermediate/current_annotation_file.csv")
# defining the label variable, with a possible recategorization before training
label_categories = {
    "Poubelle" : "Anything but personne",
    "Humain comme empathie" : "Anything but personne",
    "Genre/espèce humaine": "Anything but personne",
    "Matériel biologique": "Anything but personne",
    "Individu": "Anything but personne",
    "Agent responsable":"Personne",
    "Personne relationnelle": "Personne",
    "Personne à protéger":"Personne"

}

if(simplify_labels):
    annotations_file['label'] = annotations_file['Personne'].map(label_categories)
else:
    annotations_file['label'] = annotations_file['Personne']

# defining text variable    
annotations_file['text'] = annotations_file['sentence_text']


annotations_file.sample(5)

,sentence_id,Personne,sentence_text,label,text
393,1140012,Individu,DGT : Direction générale du travail. DRH : Dir...,Individu,DGT : Direction générale du travail. DRH : Dir...
1225,780242,NaN,Un dernier point concerne la question de la « ...,NaN,Un dernier point concerne la question de la « ...
848,590401,NaN,"III - Groupe ""Prospective personnes âgées""\nCo...",NaN,"III - Groupe ""Prospective personnes âgées""\nCo..."
187,1270324,Personne relationnelle,La variété des flux amène en France des res so...,Personne relationnelle,La variété des flux amène en France des res so...
589,1290295,Agent responsable,Elle crée notamment la Commission \nnationale ...,Agent responsable,Elle crée notamment la Commission \nnationale ...


In [4]:
# filtering out not annotated (na) and some categories
annotations = annotations_file.dropna()
annotations = annotations[~annotations["Personne"].isin(["Humain comme empathie", "Poubelle","Personne à protéger","Matériel biologique"])]
annotations.sample(5)

,sentence_id,Personne,sentence_text,label,text
322,1280420,Personne relationnelle,L’expérimentati on de telles dynamiques \ndans...,Personne relationnelle,L’expérimentati on de telles dynamiques \ndans...
575,1400158,Individu,1.2.1. Un creusement des inégalités sociales e...,Individu,1.2.1. Un creusement des inégalités sociales e...
246,1360339,Agent responsable,"Depuis la \nloi Caillavet du 22 décembre 1976,...",Agent responsable,"Depuis la \nloi Caillavet du 22 décembre 1976,..."
237,1260319,Individu,La société n’a pas pour seule fonction de prom...,Individu,La société n’a pas pour seule fonction de prom...
262,770427,Agent responsable,Cet\nintérêt englobe la connaissance génétique...,Agent responsable,Cet\nintérêt englobe la connaissance génétique...


In [5]:
annotations["Personne"].value_counts()

Personne
Individu                  118
Agent responsable         104
Personne relationnelle     97
Genre/espèce humaine       90
Name: count, dtype: int64

In [6]:
annotations["label"].value_counts()

label
Individu                  118
Agent responsable         104
Personne relationnelle     97
Genre/espèce humaine       90
Name: count, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.80)

In [8]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [9]:
batch_size = 16
train_loader = bert.encode(train_data.text.values,
train_data.label.values, batch_size=batch_size)

  0%|          | 0/327 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/327 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Individu': 2, 'Personne relationnelle': 3}


In [10]:
test_loader = bert.encode(test_data.text.values,
test_data.label.values, batch_size=batch_size)

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Individu': 2, 'Personne relationnelle': 3}


In [11]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=3,
    lr=2e-5,
    random_state=42,
    save_model_as='clickbait'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 3 ========
Training...

  Average training loss: 1.38
  Training took: 0:00:31

Running Validation...

  Average test loss: 1.35
  Validation took: 0:00:02
                        precision    recall  f1-score   support

     Agent responsable       0.00      0.00      0.00        30
  Genre/espèce humaine       0.00      0.00      0.00        17
              Individu       0.26      1.00      0.41        21
Personne relationnelle       0.00      0.00      0.00        14

              accuracy                           0.26        82
             macro avg       0.06      0.25      0.10        82
          weighted avg       0.07      0.26      0.10        82


======== Epoch 2 / 3 ========
Training...


/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/m


  Average training loss: 1.35
  Training took: 0:00:31

Running Validation...

  Average test loss: 1.34
  Validation took: 0:00:02
                        precision    recall  f1-score   support

     Agent responsable       0.00      0.00      0.00        30
  Genre/espèce humaine       0.00      0.00      0.00        17
              Individu       0.27      1.00      0.42        21
Personne relationnelle       0.33      0.07      0.12        14

              accuracy                           0.27        82
             macro avg       0.15      0.27      0.13        82
          weighted avg       0.12      0.27      0.13        82


======== Epoch 3 / 3 ========
Training...


/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/m


  Average training loss: 1.34
  Training took: 0:00:33

Running Validation...

  Average test loss: 1.33
  Validation took: 0:00:02
                        precision    recall  f1-score   support

     Agent responsable       0.00      0.00      0.00        30
  Genre/espèce humaine       0.00      0.00      0.00        17
              Individu       0.27      1.00      0.42        21
Personne relationnelle       0.25      0.07      0.11        14

              accuracy                           0.27        82
             macro avg       0.13      0.27      0.13        82
          weighted avg       0.11      0.27      0.13        82


Training complete!


/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/m

In [12]:
import numpy as np
pred_data = annotations_file[annotations_file["label"].isna()][["text"]].dropna()
pred_loader = bert.encode(pred_data.text.values, batch_size=batch_size)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Individu': 2, 'Personne relationnelle': 3}


In [13]:
pred_data["pred_proba"].describe()

count    798.000000
mean       0.290116
std        0.011629
min        0.259866
25%        0.281881
50%        0.288847
75%        0.298296
max        0.325199
Name: pred_proba, dtype: float64

In [14]:
pd.set_option('display.max_colwidth', None)  # Use None to show all the text, or set a large number

pred_data

,text,pred_label,pred_proba
700,"S’il semble dès lors naïf de penser que le sentiment de soi est dépendant \nd’une augmentation ou d’une diminution quelconque, on ne peut en revanche exclure que ce sentiment \npuisse être modifié quand un seuil technique, par la prise régulière et prolongée de médicaments \nnotamme nt, est franchi. Il importe donc de tenir compte de toutes les modifications du comportement \nd’une personne quand on veut évaluer les processus dits « internes » liés au sentiment de soi 91. En raison du manque d’études d’observation au long cours sur la ne uroamélioration (IIIb), la \nquestion reste posée du risque éventuel de modification de capacités premières, relatives au soi, de la \npersonne humaine comme « être en bonne santé, vivre longtemps, avoir une bonne mémoire, être \nd’une humeur égale, ne pas être durablement dans des états affectifs paralysants comme la timidité ou \nla peur » (B.Baertschi, 2011).",2,0.295188
701,"Personne ne soutient qu'il soit possible de tourner ce droit des parents pour entreprendre\ndes recherches sur l'embryon. Mais, ici encore, la question primaire, celle du principe de\ntelles recherches, suscitera sans doute des oppositions passionnées entre ceux qui\nvoudraient rendre les recherches impossibles et ceux qui n'ont aucune objection à ce\nqu'elles soient menées avec l'accord des personnes qui ont donné vie à l'embryon, par des\néquipes connues et sérieuses qui en garantiraient la destination. Par ailleurs, il ne saurait être question de refuser un statut juridique aux enfants nés d'une\nfécondation artificielle.",2,0.277055
702,"Le temps de parole doit être suffisant pour \nque le mineur éventuel puisse, dans une relation avec l’expert faite de confiance, d’ouverture \nd’esprit et de dialogue, saisir non seulement les raisons de la demande d’examen mais en \nmême temps le caractère aléatoire de ses résultats. 4 - Ces examens médicaux sont actuellement pratiqués en l’absence de consentement de la \npersonne ellemême ou d’un tuteur ou d’une personne de référence. Ils ne peuvent être \nprescrits et réalisés que dans le respect attentif de la personne qui se réclame du statut de \nmineur.",3,0.279503
703,"»\n24 Avis N°107 du 15 octobre 2009 sur les problèmes éthiques liés aux diagnostics anténatals : le diagnostic \nprénatal (DPN) et le diagnostic préimplantatoire (DPI) . 25 Dans l’Avis N°8 du 15 décembre 1986 relatif aux recherches et utilisation des embryons humains in vitro à des \nfins médicales et scientifiques : « Le Comité constate que le développement de la procréation par la fécondation \nin vitro accentue la tendance à réduire les corps humains à l’état d’instruments ; de plus, des techniques telles \nque la congélation des embryons renforcent le caractère artificiel de la reproduction, notamment par la \ndissociation entre la fécondation et la gestation. […] On peut envisager et souhaiter que la recherche permette à \nl’avenir de ne féconder que des ovocytes destinés à être transplantés en vue de la naissance d’un enfant.",2,0.295519
704,"Sur la question du champ d'application de la loi, ils recommandent en\nl'argumentant, "" l'extension du champ de la loi à toute recherche scientifique sur\nl'être humain"" . N° 41 : La Coopération dans le domaine de la recherche biomédicale entre équipes\nfrançaises et équipes de pays en voie de développement économique, 1993\nConsidérant que "" les projets de recherche biomédicale entre les équipes françaises et les\néquipes de pays en voie de développement économique exigent des précautions\nparticulières afin que soient garanties la dignité et la sécurité des personnes se prêtant à\nces recherches d'intérêt général"" , le CCNE déclare : "" Tous les projets doivent être\nsoumis, avant que d'être entrepris, à un Comité spécialisé à créer (Comité\nconsultatif français de protection des personnes se prêteant à la recherche\nbiomédicale dans les pays en voie de développement, ou CCPPVD). Ce Comité spécialisé doit notamment inclure des